In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
file_path = 'final_data.csv'

stock_data = pd.read_csv(file_path)

In [3]:
# Check for and drop the 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in stock_data.columns:
    stock_data.drop(columns=['Unnamed: 0'], inplace=True)
    
stock_data

,Adj Close,Volume,Date,Symbol,Year,Outstanding_Shares
0,429.1,1411814,01/01/2014,MCX,2014,50.998
1,424.3,2919045,02/01/2014,MCX,2014,50.998
2,499.6,8191055,03/01/2014,MCX,2014,50.998
3,497.8,8395828,06/01/2014,MCX,2014,50.998
4,517.0,6823517,07/01/2014,MCX,2014,50.998
...,...,...,...,...,...,...
830700,2565.1,8270892,22/12/2023,RELIANCE,2023,6765.684
830701,2578.1,3732832,26/12/2023,RELIANCE,2023,6765.684
830702,2586.9,4602078,27/12/2023,RELIANCE,2023,6765.684
830703,2605.6,6151318,28/12/2023,RELIANCE,2023,6765.684


In [4]:
# Convert 'Outstanding_Shares' to numeric (if not already)
stock_data['Outstanding_Shares'] = pd.to_numeric(stock_data['Outstanding_Shares'], errors='coerce')

# Multiply 'Outstanding_Shares' by 10,000,000 (since it's in crores) before calculating turnover
stock_data['Outstanding_Shares'] *= 10000000

# Calculate daily turnover
stock_data['Turnover'] = stock_data['Volume'] / stock_data['Outstanding_Shares']

# Display the first few rows with the new Turnover column
stock_data.head()

,Adj Close,Volume,Date,Symbol,Year,Outstanding_Shares,Turnover
0,429.1,1411814,01/01/2014,MCX,2014,509980000.0,0.002768
1,424.3,2919045,02/01/2014,MCX,2014,509980000.0,0.005724
2,499.6,8191055,03/01/2014,MCX,2014,509980000.0,0.016062
3,497.8,8395828,06/01/2014,MCX,2014,509980000.0,0.016463
4,517.0,6823517,07/01/2014,MCX,2014,509980000.0,0.013380


In [6]:
# Convert 'Date' to datetime format
stock_data['Date'] = pd.to_datetime(stock_data['Date'], format="%d/%m/%Y")

# Set 'Date' as the index
stock_data.set_index('Date', inplace=True)

In [7]:
# Resample to monthly frequency to get the first price of each month
monthly_data = stock_data.groupby('Symbol').resample('BMS').first()

# Reset the index to make 'Symbol' and 'Date' columns again
monthly_data = monthly_data.reset_index(level=0, drop=True)

# Calculate monthly returns
monthly_data['Monthly_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change()

# Calculate cumulative returns for 3, 6, 9, and 12 months
monthly_data['3M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(3)
monthly_data['6M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(6)
monthly_data['9M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(9)
monthly_data['12M_Return'] = monthly_data.groupby('Symbol')['Adj Close'].pct_change(12)


monthly_data.reset_index(inplace=True)

In [8]:
# Rank stocks based on cumulative returns
for period in ['3M', '6M', '9M', '12M']:
    rank_col = f'{period}_Rank'
    monthly_data[rank_col] = monthly_data.groupby('Date')[f'{period}_Return'].rank(method='first', ascending=False)

In [9]:
# Define quintile function with check for sufficient unique values
def assign_quintile(x, rank_col, quintile_col):
    if len(x[rank_col].unique()) < 5:
        x[quintile_col] = np.nan
    else:
        x[quintile_col] = pd.qcut(x[rank_col], 5, labels=False) + 1
    return x

# Apply quintile ranking
for period in ['3M', '6M', '9M', '12M']:
    rank_col = f'{period}_Rank'
    quintile_col = f'{period}_Quintile'
    monthly_data = monthly_data.groupby('Date').apply(assign_quintile, rank_col, quintile_col).reset_index(drop=True)


In [11]:
monthly_data[monthly_data['Symbol'] == '3MINDIA']

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,9M_Return,12M_Return,3M_Rank,6M_Rank,9M_Rank,12M_Rank,3M_Quintile,6M_Quintile,9M_Quintile,12M_Quintile
0,2014-01-01,3445.1,149,3MINDIA,2014,112650000.0,1.322681e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,2014-02-03,3443.1,76,3MINDIA,2014,112650000.0,6.746560e-07,-0.000581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
674,2014-03-03,3461.4,419,3MINDIA,2014,112650000.0,3.719485e-06,0.005315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,2014-04-01,3404.7,253,3MINDIA,2014,112650000.0,2.245894e-06,-0.016381,-0.011727,NaN,NaN,NaN,253.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
1348,2014-05-01,3550.6,239,3MINDIA,2014,112650000.0,2.121616e-06,0.042853,0.031222,NaN,NaN,NaN,293.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38755,2023-08-01,28210.7,14027,3MINDIA,2023,112650000.0,1.245184e-04,0.053841,0.281466,0.267014,0.286040,0.297062,88.0,140.0,130.0,149.0,2.0,3.0,2.0,3.0
39092,2023-09-01,30737.6,2486,3MINDIA,2023,112650000.0,2.206835e-05,0.089572,0.189094,0.379865,0.336731,0.414146,151.0,111.0,108.0,117.0,3.0,2.0,2.0,2.0
39429,2023-10-02,30959.9,1746,3MINDIA,2023,112650000.0,1.549933e-05,0.007232,0.156541,0.389034,0.464178,0.343005,145.0,135.0,89.0,160.0,3.0,2.0,2.0,3.0
39766,2023-11-01,29918.2,2663,3MINDIA,2023,112650000.0,2.363959e-05,-0.033647,0.060527,0.359029,0.343702,0.363880,133.0,99.0,138.0,124.0,2.0,2.0,3.0,2.0


In [12]:
# Calculate average turnover over the formation periods
for period in ['3M', '6M', '9M', '12M']:
    avg_turnover_col = f'{period}_Avg_Turnover'
    window_size = int(period[:-1])
    monthly_data[avg_turnover_col] = monthly_data.groupby('Symbol')['Turnover'].rolling(window=window_size,min_periods=1).mean().reset_index(level=0, drop=True)

In [13]:
monthly_data[monthly_data['Symbol'] == '3MINDIA']

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,...,9M_Rank,12M_Rank,3M_Quintile,6M_Quintile,9M_Quintile,12M_Quintile,3M_Avg_Turnover,6M_Avg_Turnover,9M_Avg_Turnover,12M_Avg_Turnover
0,2014-01-01,3445.1,149,3MINDIA,2014,112650000.0,1.322681e-06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.322681e-06,1.322681e-06,1.322681e-06,1.322681e-06
337,2014-02-03,3443.1,76,3MINDIA,2014,112650000.0,6.746560e-07,-0.000581,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.986684e-07,9.986684e-07,9.986684e-07,9.986684e-07
674,2014-03-03,3461.4,419,3MINDIA,2014,112650000.0,3.719485e-06,0.005315,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.905607e-06,1.905607e-06,1.905607e-06,1.905607e-06
1011,2014-04-01,3404.7,253,3MINDIA,2014,112650000.0,2.245894e-06,-0.016381,-0.011727,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,2.213345e-06,1.990679e-06,1.990679e-06,1.990679e-06
1348,2014-05-01,3550.6,239,3MINDIA,2014,112650000.0,2.121616e-06,0.042853,0.031222,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,2.695665e-06,2.016866e-06,2.016866e-06,2.016866e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38755,2023-08-01,28210.7,14027,3MINDIA,2023,112650000.0,1.245184e-04,0.053841,0.281466,0.267014,...,130.0,149.0,2.0,3.0,2.0,3.0,7.742861e-05,4.970410e-05,4.212063e-05,3.827785e-05
39092,2023-09-01,30737.6,2486,3MINDIA,2023,112650000.0,2.206835e-05,0.089572,0.189094,0.379865,...,108.0,117.0,3.0,2.0,2.0,2.0,5.666519e-05,4.896138e-05,4.242048e-05,3.851013e-05
39429,2023-10-02,30959.9,1746,3MINDIA,2023,112650000.0,1.549933e-05,0.007232,0.156541,0.389034,...,89.0,160.0,3.0,2.0,2.0,3.0,5.402870e-05,4.835627e-05,3.964393e-05,3.623317e-05
39766,2023-11-01,29918.2,2663,3MINDIA,2023,112650000.0,2.363959e-05,-0.033647,0.060527,0.359029,...,138.0,124.0,2.0,2.0,3.0,2.0,2.040243e-05,4.891552e-05,3.993687e-05,3.669108e-05


In [14]:
# Rank stocks based on average turnover
for period in ['3M', '6M', '9M', '12M']:
    avg_turnover_col = f'{period}_Avg_Turnover'
    rank_col = f'{period}_Turnover_Rank'
    monthly_data[rank_col] = monthly_data.groupby('Date')[avg_turnover_col].rank(method='first')

In [15]:
# Define tercile function with check for sufficient unique values
def assign_tercile(x, rank_col, tercile_col):
    if len(x[rank_col].unique()) < 3:
        x[tercile_col] = np.nan
    else:
        x[tercile_col] = pd.qcut(x[rank_col], 3, labels=False) + 1
    return x

# Apply tercile ranking
for period in ['3M', '6M', '9M', '12M']:
    rank_col = f'{period}_Turnover_Rank'
    tercile_col = f'{period}_Turnover_Tercile'
    monthly_data = monthly_data.groupby('Date').apply(assign_tercile, rank_col, tercile_col).reset_index(drop=True)

In [16]:
monthly_data[monthly_data['Date'] == '2014-10-01']

,Date,Adj Close,Volume,Symbol,Year,Outstanding_Shares,Turnover,Monthly_Return,3M_Return,6M_Return,...,9M_Avg_Turnover,12M_Avg_Turnover,3M_Turnover_Rank,6M_Turnover_Rank,9M_Turnover_Rank,12M_Turnover_Rank,3M_Turnover_Tercile,6M_Turnover_Tercile,9M_Turnover_Tercile,12M_Turnover_Tercile
3033,2014-10-01,6023.7,453,3MINDIA,2014,1.126500e+08,0.000004,0.053610,0.407242,0.769231,...,0.000003,0.000003,4.0,3.0,3.0,3.0,1,1,1,1
3034,2014-10-01,64.2,122064,AARTIIND,2014,3.332820e+09,0.000037,0.011024,0.337500,1.284698,...,0.000101,0.000094,68.0,138.0,151.0,149.0,1,2,2,2
3035,2014-10-01,963.5,63905,ABB,2014,2.119080e+09,0.000030,0.025873,-0.032242,0.314820,...,0.000088,0.000083,136.0,108.0,136.0,130.0,2,1,2,2
3036,2014-10-01,3059.8,3557,ABBOTINDIA,2014,2.124900e+08,0.000017,0.261825,0.495650,0.845810,...,0.000015,0.000014,11.0,7.0,12.0,12.0,1,1,1,1
3037,2014-10-01,123.2,12477,ABFRL,2014,9.413900e+08,0.000013,0.094139,0.005714,0.522868,...,0.000022,0.000021,43.0,33.0,23.0,23.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365,2014-10-01,104.7,5309380,YESBANK,2014,2.096607e+10,0.000253,-0.065179,0.002874,0.379447,...,0.000992,0.000925,312.0,322.0,330.0,327.0,3,3,3,3
3366,2014-10-01,295.7,3041803,ZEEL,2014,9.604490e+09,0.000317,0.101714,0.053813,0.154627,...,0.000241,0.000221,227.0,237.0,249.0,245.0,3,3,3,3
3367,2014-10-01,106.8,628350,ZENSARTECH,2014,2.227050e+09,0.000282,0.360510,0.449118,0.658385,...,0.000081,0.000094,191.0,145.0,124.0,148.0,2,2,2,2
3368,2014-10-01,3847.7,3769,ZFCVINDIA,2014,1.896800e+08,0.000020,0.076340,0.172007,0.932546,...,0.000018,0.000016,18.0,14.0,17.0,15.0,1,1,1,1


In [19]:
# Initialize a DataFrame to store portfolio returns
portfolio_returns = pd.DataFrame()

# Calculate monthly returns for intersection portfolios
for period in ['3M', '6M', '9M', '12M']:
    for r in range(1, 6):
        for t in range(1, 4):
            portfolio_label = f'{period}_R{r}_TO{t}'
            mask = (monthly_data[f'{period}_Quintile'] == r) & (monthly_data[f'{period}_Turnover_Tercile'] == t)
            portfolio_returns[portfolio_label] = monthly_data[mask].groupby('Date')['Monthly_Return'].mean()

In [20]:
portfolio_returns

,3M_R1_TO1,3M_R1_TO2,3M_R1_TO3,3M_R2_TO1,3M_R2_TO2,3M_R2_TO3,3M_R3_TO1,3M_R3_TO2,3M_R3_TO3,3M_R4_TO1,...,12M_R2_TO3,12M_R3_TO1,12M_R3_TO2,12M_R3_TO3,12M_R4_TO1,12M_R4_TO2,12M_R4_TO3,12M_R5_TO1,12M_R5_TO2,12M_R5_TO3
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-01,0.177807,0.198763,0.223997,0.131660,0.119910,0.193441,0.134978,0.122044,0.117816,0.085508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-01,0.259154,0.137803,0.155644,0.056240,0.079495,0.045389,0.041771,0.057365,-0.010362,0.026239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-02,0.439785,0.451664,0.466845,0.292003,0.273594,0.305766,0.172634,0.185834,0.168467,0.132562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-07-01,0.327234,0.287888,0.213714,0.132687,0.095971,0.158173,0.103911,0.076968,0.065737,0.074931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-01,0.042207,0.055762,0.007734,0.030672,-0.065855,-0.015634,0.006207,-0.061917,-0.057282,0.001081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.116930,0.212680,0.168803,0.077696,0.110988,0.100531,0.036624,0.072246,0.076667,0.031986,...,0.077929,0.047995,0.063345,0.066497,0.046794,0.043352,0.051145,-0.004118,0.039880,0.058824
2023-09-01,0.188636,0.197287,0.185223,0.097432,0.041609,0.112508,0.029969,0.025082,0.066423,0.001319,...,0.110614,0.029265,0.017075,0.045877,0.014913,0.018700,0.045409,0.017835,0.023386,0.032864
2023-10-02,0.133515,0.147399,0.103557,0.072677,0.029309,0.035737,0.031432,0.023559,-0.012934,0.008393,...,0.015505,0.034437,0.019055,0.026759,0.012403,-0.000434,0.010042,-0.014305,0.001831,0.008812


In [21]:
portfolio_returns.to_csv('portfolio_returns.csv')